# GISY6400 Capstone - Wildfire Hazard: Terrain Mapping

Program: wildfire_terrain.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook produces terrain products for wildfire hazard mapping (hillshade, slope, aspect, elevation)  
Date: May 4, 2025

### Import python modules

In [1]:
import arcpy
from arcpy.sa import *
import os

### Paths & Workspace Setup

In [2]:
# Get project directory
aprx = arcpy.mp.ArcGISProject("CURRENT")
gis_dir = os.path.dirname(aprx.filePath)
project_dir = os.path.dirname(gis_dir)

# Directory structure
raw_dir = os.path.join(project_dir, "data", "raw")
extract_dir = os.path.join(project_dir, "data", "extracted")
proc_dir = os.path.join(project_dir, "data", "processed")
land_mask_path = os.path.join(project_dir, "gis", "capstone.gdb", "hydro_la_poly")
 
# Create folders if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)

In [3]:
# Set ArcPy Workspace & overwrite options
arcpy.env.workspace = os.path.join(project_dir, "gis", "capstone.gdb")
arcpy.env.overwriteOutput = True

# Activate Spatial Analyst License
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [4]:
# Define output file paths
hillshade_output = os.path.join(proc_dir, "hillshade.tif")
elev_output = os.path.join(proc_dir, "elev_hazard_map.tif")
slope_output = os.path.join(proc_dir, "slope_hazard_map.tif")
aspect_output = os.path.join(proc_dir, "aspect_hazard_map.tif")
terrain_hazard_output = os.path.join(proc_dir, "terrain_hazard_map.tif")

# Define input raster & handle no data values
dem_utm_clipped_path = os.path.join(proc_dir, "dem_centre_west_utm_clipped.tif")
input_raster = Raster(dem_utm_clipped_path)
input_raster_filtered = SetNull(IsNull(input_raster), input_raster)
input_raster_filtered = Raster(input_raster_filtered)

### Hillshade

In [ ]:
# Create hillshade layer
hillshade = arcpy.sa.Hillshade(
    in_raster=dem_utm_clipped_path,
    azimuth=315,
    altitude=45,
    model_shadows="SHADOWS",
    z_factor=1
)

# Save output
hillshade.save(hillshade_output)
print(f"Hillshade saved to: {hillshade_output}")

### Elevation

In [ ]:
# Classify elevation thresholds based on wildfire behaviour
elev_hazard_map = Con(
    (input_raster_filtered < 100), 5, # Very High (up to 100m)
    Con(
        (input_raster_filtered >= 100) & (input_raster_filtered < 200), 4, # High (100-200m)
        Con(
            (input_raster_filtered >= 200) & (input_raster_filtered < 300), 3, # Medium (200-300m)
            Con(
                (input_raster_filtered >= 300) & (input_raster_filtered < 400), 2, # Low (300-400m)
                Con(
                    input_raster_filtered >= 400, 1 # Very Low (400+m)
                )
            )
        )
    )
)

# Mask elevation hazard map by land area
elev_hazard_map_masked = ExtractByMask(elev_hazard_map, land_mask_path)

# Save the classified elevation raster
elev_hazard_map_masked.save(elev_output)
print(f"Elevation hazard map saved to: {elev_output}")

### Slope

In [ ]:
# Calculate slope in degrees
slope_raster = Slope(input_raster_filtered, output_measurement="DEGREE")

# Classify slope thresholds based on wildfire behaviour
slope_hazard_map = Con(
    slope_raster < 5, 1, # Low (<5 Deg)
    Con(
        (slope_raster >= 5) & (slope_raster < 20), 2, # Moderate (5-20 Deg)
        Con(
            (slope_raster >= 20) & (slope_raster < 35), 3, # High (20-35 Deg)
            Con(
                slope_raster >= 35, 4) # Very High (>= 35 Deg)
        )
    )
)

# Mask slope hazard map by land area
slope_hazard_map_masked = ExtractByMask(slope_hazard_map, land_mask_path)

# Save the classified slope raster
slope_hazard_map_masked.save(slope_output)
print(f"Slope hazard map saved to: {slope_output}")

### Aspect

In [ ]:
# Compute Aspect from DEM
aspect = Aspect(input_raster_filtered)

# Classify aspect thresholds based on wildfire behaviour
aspect_hazard_map = Con(
    (aspect < 45) | (aspect > 315), 1, # Low (North)
    Con(
        (aspect >= 45) & (aspect < 135), 2, # Medium (East)
        Con(
            (aspect >= 135) & (aspect <= 225), 4, # Very High (South) 
            Con(
                (aspect > 225) & (aspect <= 315), 3) # High (West)
        )
    )
)

# Mask the aspect hazard map by the land area
aspect_hazard_map_masked = ExtractByMask(aspect_hazard_map, land_mask_path)

# Save the final aspect hazard map
aspect_hazard_map_masked.save(aspect_output)
print(f"Aspect hazard map saved to: {aspect_output}")

### Composite Terrain Hazard Map

In [5]:
# Load the individual hazard rasters
elev_hazard_masked = Raster(elev_output)
slope_hazard_masked = Raster(slope_output)
aspect_hazard_masked = Raster(aspect_output)

# Weighted sum
terrain_hazard = (elev_hazard_masked * 0.25) + (slope_hazard_masked * 0.4) + (aspect_hazard_masked * 0.35)

# Get min and max values from terrain_hazard
min_val = float(arcpy.GetRasterProperties_management(terrain_hazard, "MINIMUM").getOutput(0))
max_val = float(arcpy.GetRasterProperties_management(terrain_hazard, "MAXIMUM").getOutput(0))

# Normalize to a 1–5 scale
normalized = ((terrain_hazard - min_val) / (max_val - min_val)) * 4 + 1

# Classify into 5 categories based on normalized value
terrain_hazard_classified = Con(
    normalized < 1.5, 1,
    Con(normalized < 2.5, 2,
    Con(normalized < 3.5, 3,
    Con(normalized < 4.5, 4, 5))))


# Save the final raster
terrain_hazard_classified.save(terrain_hazard_output)
print(f"5-class terrain wildfire hazard map saved to: {terrain_hazard_output}")

5-class terrain wildfire hazard map saved to: D:\Dropbox\COGS\Capstone\data\processed\terrain_hazard_map.tif


In [ ]:
print(f"Min: {terrain_hazard_classified.minimum}, Max: {terrain_hazard.maximum}")
